import modules

In [1]:
import pandas as pd
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: /Users/haph/Study/ubion/Project1/mid_code


load data

In [2]:
cny = pd.read_csv('../dataset/CNY_KRW.csv')[['날짜','종가','시가']].rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open'})

In [3]:
# cny['cny_yesterday_change'] = round((cny['close']-cny['open'])/cny['open']*100, 3)
# cny['cny_yesterday_change'] = cny['cny_yesterday_change'].shift(1)

# # 파생변수 생성(야간 변동율 = night_change)
# cny['close_shift'] = cny['close'].shift(1)
# cny['cny_night_change'] = round((cny['open']-cny['close_shift'])/cny['close_shift']*100, 3)

In [4]:
cny.head(10)

,date,close,open
0,2023- 10- 27,184.98,184.72
1,2023- 10- 26,184.88,184.89
2,2023- 10- 25,184.81,183.90
3,2023- 10- 24,183.82,183.66
4,2023- 10- 23,183.45,184.57
5,2023- 10- 20,184.37,185.57
6,2023- 10- 19,185.39,185.29
7,2023- 10- 18,185.06,185.10
8,2023- 10- 17,184.96,184.41
9,2023- 10- 16,184.36,185.21


In [5]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    66 non-null     object 
 1   close   66 non-null     float64
 2   open    66 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.7+ KB


기간 통일

In [6]:
ks200 = pd.read_csv('../dataset/clean_feature/ks200_present.csv', index_col=0)

In [7]:
# 데이터 타입 변경
cny['date'] = pd.to_datetime(cny['date'])

In [8]:
ks200.date = pd.to_datetime(ks200.date)

In [9]:
cny = pd.merge(cny, ks200, left_on='date', right_on='date', how='inner')[cny.columns]

#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [10]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
cny['cny_yesterday_change'] = round((cny['close']-cny['open'])/cny['open']*100, 3)
cny['cny_yesterday_change'] = cny['cny_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
cny['close_shift'] = cny['close'].shift(1)
cny['cny_night_change'] = round((cny['open']-cny['close_shift'])/cny['close_shift']*100, 3)

cny.drop('close_shift',axis=1,inplace=True)

In [11]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
# cny = cny.loc[cny['date']<='2023-10-20',].loc[cny['date']>='2023-07-31',]
cny = cny[['date','cny_yesterday_change','cny_night_change']]

In [12]:
# 날짜순으로 오름차순 정렬
cny.sort_values(by='date', inplace=True)
cny.reset_index(drop=True, inplace=True)

In [13]:
cny.head()

,date,cny_yesterday_change,cny_night_change
0,2023-07-31,0.346,-0.824
1,2023-08-01,0.534,-0.908
2,2023-08-02,0.304,-0.883
3,2023-08-03,0.248,-0.638
4,2023-08-04,-0.093,-0.176


In [14]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  54 non-null     datetime64[ns]
 1   cny_yesterday_change  53 non-null     float64       
 2   cny_night_change      53 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.4 KB


In [15]:
cny

,date,cny_yesterday_change,cny_night_change
0,2023-07-31,0.346,-0.824
1,2023-08-01,0.534,-0.908
2,2023-08-02,0.304,-0.883
3,2023-08-03,0.248,-0.638
4,2023-08-04,-0.093,-0.176
5,2023-08-07,0.302,-0.274
6,2023-08-08,-0.219,-0.132
7,2023-08-09,0.154,0.038
8,2023-08-10,0.619,-0.882
9,2023-08-11,0.218,-0.874


In [16]:
cny['cny_yesterday_change'].loc[cny['date'] == '2023-10-20'] = -0.607
cny['cny_night_change'].loc[cny['date'] == '2023-10-20'] = 1.156

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_90116/342973682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cny['cny_yesterday_change'].loc[cny['date'] == '2023-10-20'] = -0.607
/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_90116/342973682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cny['cny_night_change'].loc[cny['date'] == '2023-10-20'] = 1.156


In [17]:
cny

,date,cny_yesterday_change,cny_night_change
0,2023-07-31,0.346,-0.824
1,2023-08-01,0.534,-0.908
2,2023-08-02,0.304,-0.883
3,2023-08-03,0.248,-0.638
4,2023-08-04,-0.093,-0.176
5,2023-08-07,0.302,-0.274
6,2023-08-08,-0.219,-0.132
7,2023-08-09,0.154,0.038
8,2023-08-10,0.619,-0.882
9,2023-08-11,0.218,-0.874


csv 파일로 새로 저장

In [18]:
cny.to_csv("../dataset/clean_feature/cny_krw_final.csv")